# Setup Log Tables

This notebook creates the required log tables for Bronze and Silver processing.

**Important:** This notebook only needs to be run once. The `ensure_log_tables()` function is idempotent and safe to call multiple times.

## Log Tables Created:

1. **logs.bronze_processing_log** - Individual Bronze table processing records (partitioned by run_date, table_name)
2. **logs.bronze_run_summary** - Bronze run aggregate statistics
3. **logs.silver_processing_log** - Individual Silver CDC merge records (partitioned by run_date)
4. **logs.silver_run_summary** - Silver run aggregate statistics

## Parameters

In [ ]:
# Parameters
log_to_console = True
debug_mode = True  # Set to True for detailed logging during table creation

## Imports and Spark Session

In [ ]:
# Module fabric.bootstrap
# ---------------------
# This cell enables a flexible module loading strategy:
#
# PRODUCTION (default): The `Files/code` directory is empty. This function does nothing,
# and Python imports all modules from the stable, versioned Wheel in the Environment.
#
# DEVELOPMENT / HOTFIX: To bypass the 15-20 minute Fabric publish cycle for urgent fixes,
# upload individual .py files to `Files/code` in the Lakehouse. This function prepends
# that path to sys.path, so Python finds the override files first. All other modules
# continue to load from the Wheel - only the uploaded files are replaced.
#
# Usage: Keep `Files/code` empty for production stability. Use it only for rapid
# iteration during development or emergency hotfixes.

from modules.fabric_bootstrap import ensure_module_path
ensure_module_path()  # Now Python can find the rest

In [ ]:
from modules.logging_utils import configure_logging, ensure_log_tables
from modules.spark_session import get_or_create_spark_session
from modules.path_utils import detect_environment
import logging

# Configure file logging
log_file = configure_logging(
    run_name="setup_log_tables",
    enable_console_logging=log_to_console
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.info(f"Logfile: {log_file}")

In [ ]:
# Initialize Spark session
spark = get_or_create_spark_session(
    app_name="Setup_Log_Tables",
    enable_hive=True
)

# Detect environment
environment = detect_environment(spark)
logger.info(f"Detected environment: {environment}")

## Create Log Tables

The `ensure_log_tables()` function:
- Checks if tables already exist
- Creates them only if they don't exist
- Is safe to run multiple times
- Uses the correct partitioning strategy for each table

In [ ]:
try:
    logger.info("Starting log table initialization...")
    ensure_log_tables(spark, debug=debug_mode)
    logger.info("✓ All log tables are ready!")
    print("\n" + "="*60)
    print("SUCCESS: Log tables setup completed")
    print("="*60)
except Exception as e:
    logger.error(f"✗ Failed to setup log tables: {e}")
    print("\n" + "="*60)
    print(f"ERROR: {e}")
    print("="*60)
    raise

## Verification

Verify that all tables were successfully created.

In [ ]:
# Verify tables exist
log_tables = [
    "logs.bronze_processing_log",
    "logs.bronze_run_summary",
    "logs.silver_processing_log",
    "logs.silver_run_summary"
]

print("\nVerifying log tables:")
print("-" * 60)

all_exist = True
for table in log_tables:
    exists = spark.catalog.tableExists(table)
    status = "✓" if exists else "✗"
    print(f"{status} {table}: {'EXISTS' if exists else 'NOT FOUND'}")
    if not exists:
        all_exist = False

print("-" * 60)

if all_exist:
    print("\n✓ All log tables verified successfully!")
    logger.info("All log tables verified successfully")
else:
    print("\n✗ Some log tables are missing!")
    logger.error("Some log tables are missing")

## View Table Schemas (Optional)

Display the schemas of the created tables.

In [ ]:
# Display schema for each table (optional)
for table in log_tables:
    if spark.catalog.tableExists(table):
        print(f"\n{'='*60}")
        print(f"Schema: {table}")
        print(f"{'='*60}")
        spark.table(table).printSchema()
        
        # Show table details
        print(f"\nTable details:")
        spark.sql(f"DESCRIBE EXTENDED {table}").filter(
            "col_name IN ('Type', 'Provider', 'Location', 'Partition Provider')"
        ).show(truncate=False)

## Test Logging (Optional)

Test if log functions work correctly by writing a test record.

In [ ]:
# Optional: Test logging functions
from modules.logging_utils import log_summary, log_batch
from datetime import datetime, date

test_enabled = False  # Set to True to run test

if test_enabled:
    logger.info("Running test log write...")
    
    # Create test summary
    test_run_ts = datetime.now().strftime("%Y%m%dT%H%M%S%f")[:-3]
    test_summary = {
        "run_ts": test_run_ts,
        "run_date": date.today(),
        "source": "test_setup",
        "status": "SUCCESS",
        "run_start": datetime.now(),
        "run_end": datetime.now(),
        "total_tables": 1,
        "tables_success": 1,
        "tables_empty": 0,
        "tables_failed": 0,
        "tables_skipped": 0,
        "total_rows": 0
    }
    
    # Write test summary
    run_log_id = log_summary(spark, test_summary, layer="bronze")
    logger.info(f"Test summary written with log_id: {run_log_id}")
    
    # Create test batch record
    test_records = [{
        "run_ts": test_run_ts,
        "run_date": date.today(),
        "run_id": f"{test_run_ts}_test",
        "source": "test_setup",
        "table_name": "test_table",
        "status": "SUCCESS",
        "rows_processed": 0,
        "start_time": datetime.now(),
        "end_time": datetime.now(),
        "duration_seconds": 0,
        "load_mode": "snapshot"
    }]
    
    # Write test batch
    log_batch(spark, test_records, layer="bronze", run_log_id=run_log_id)
    logger.info("Test batch written successfully")
    
    print("\n✓ Test logging completed successfully!")
    print(f"  - Summary log_id: {run_log_id}")
    print(f"  - Test records: {len(test_records)}")
else:
    print("\nTest logging skipped (set test_enabled=True to run test)")

## Done!

Log tables are now created and ready to use.

**Note:** From now on, all errors in `process_bronze_layer` and other notebooks will be automatically logged to these tables, even if an error occurs before the first log call.